# Training environment information

In [ ]:
!free -h

In [ ]:
!nvidia-smi -L

In [ ]:
!lscpu |grep 'Model name'

In [ ]:
!lscpu | grep 'Core(s) per socket:'

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

Preventing disconnect from Google Colab notebook:
Right-click on the connect button and paste the code to the console of the UI:


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```

# Setup

In [ ]:
!pip install fastai==2.5.3 -q
!pip freeze

In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.all import *
import sys

In [ ]:
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell
external_python_file_path="'/.'"
sys.path.append(external_python_file_path)


from eb_ml_colab_utils import get_root_path,copy_model_to_google_drive
from eb_ml_battery_lib import load_soc_dataset,generate_image_files_from_eis
from eb_ml_utils import save_model_weights,build_data_loader,build_and_train_learner,score_model

# Training Configuration

In [ ]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("batterie")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-May2022"

# List of SoC level into dataset
#config['soc_list']=['100','090','080','070','060','050','040','030','020','010']
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']


# Folder to store trained model
#config['MODELS_DIR'] = config['ROOT_DIR']+"/models"
config['MODELS_DIR'] = config['ROOT_DIR']+"/models"

Source data: battery EIS performed for SoC 100,90,80,70,60,50,40,30,20,10. Same frequnecies for each SoC

Data format:  Re{Zbat}-Im{Zbat}j

```Matlab
prefix = {...
    './Batt1/Meas1/'; --> 1
    './Batt1/Meas2/'; --> 2*
	  './Batt2/Meas1/'; --> 3  
    './Batt2/Meas2/'; --> 4*
    './Batt3/Meas1/'; --> 5
    './Batt3/Meas2/'; --> 6* ...
    './Batt4/Meas1/';
    './Batt5/Meas1/';
    './Batt6/Meas1/';
    './Batt7/Meas1/';
    './Batt8/Meas1/';
    './Batt9/Meas1/';
    './Batt10/Meas1/'; -->13
    };
```

# Data Selection

In [ ]:

# Data acquition file to load from dateset folder
all_batteries=["02_4","02_5","02_6","02_7","02_8","02_9","03_4","03_5","03_6","03_7","03_8","03_9","05_3","05_4","05_5","05_6","05_7"]#,"05_8" e batteria 6 usate per test

# List of data acquisition to use for cross validation
# A cross- validation experiment will be generate for each item in the list
cross_validation_list=[["02_4","02_5","02_6","02_7","02_8","02_9"],["03_4","03_5","03_6","03_7","03_8","03_9"],["05_3","05_4","05_5","05_6","05_7"]] #,"05_8"]] # Data acquitions on battery 6 used for testing
cross_validation_experiment_names = ["02","03","05"]

# Model Training

## Leave One Battery Out

In [ ]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_leave_one_out_"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'

model_accuracy={}
cross_validation_experiment_index = 0 # Cross validation experiment index

# Train and score model for cross-validation
for batt_measures in cross_validation_list:
  batt=cross_validation_experiment_names[cross_validation_experiment_index]
  battery_list=all_batteries.copy()
  test_battery_list = []
  for batt_mes in batt_measures:
    battery_list.remove(batt_mes)
    test_battery_list.append(batt_mes)
  print("battery data acquisitions for training and validation: ")
  print(battery_list)
  print("battery data acquisitions for test: ")
  print(test_battery_list)
  model_name=experiment_name_prefix+str(batt)
  print("model name: "+model_name)

  config['ExperimentName'] = model_name
  config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
  config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

  #GENERATE IMAGE
  if(generate_images):
      dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
      generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

      test_dataset,feature_col_names=load_soc_dataset(test_battery_list,config["soc_list"],config['DATASETS_DIR'])
      generate_image_files_from_eis(test_dataset,feature_col_names,config['TEST_IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

  # TRAINING
  learn= build_and_train_learner(config,n_epochs=n_epochs)
  #SAVE
  weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
  filename_pth= weights_filename+".pth"
  copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
  # SCORE MODEL
  dl=build_data_loader(config)
  model_accuracy[model_name]=score_model(weights_filename,dl)
  cross_validation_experiment_index+=1